In [ ]:
# ✅ Install required libraries
!pip install -U transformers datasets peft accelerate

In [ ]:
# ✅ Imports
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training
from datasets import load_dataset
import torch

In [ ]:
# ✅ Load tokenizer and base model
model_name = 'google/flan-t5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
# ✅ Prepare model for LoRA
model = prepare_model_for_int8_training(model)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=['q', 'v'],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, lora_config)

In [ ]:
# ✅ Load your dataset (adjust path if needed)
dataset = load_dataset('json', data_files='calc_dataset.jsonl', split='train')
MAX_LENGTH = 128

In [ ]:
# ✅ Preprocess function
def preprocess(example):
    inputs = tokenizer(example['prompt'], max_length=MAX_LENGTH, truncation=True, padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example['output'], max_length=MAX_LENGTH, truncation=True, padding='max_length')
    inputs['labels'] = labels['input_ids']
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=False)
print('Tokenization complete.')

In [ ]:
# ✅ Split dataset
splits = tokenized_dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)
train_dataset = splits['train']
eval_dataset = splits['test']

In [ ]:
# ✅ Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./t5_lora_xml_model',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-4,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

In [ ]:
# ✅ Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# ✅ Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
# ✅ Train!
trainer.train()